## Pubmed Abstract/Fulltext Fetching Merging and Cleaning

Pubmed does not give fulltext/abstract text in a good format. It gives a CSV with the metadata of searched results. That means you have to use the Entrez fetch and then merge the abstracts and text information with the metadata to have something workable

In [16]:
# with open("clinical_study_abst.txt") as f:
#     text = f.read()
    
df = pd.read_csv("downloads/clin_study_meta.csv")
records = get_pubmed_records([str(pmid) for pmid in df["PMID"]])
len([r for r in records["PubmedArticle"] if r is not None])

articles = records["PubmedArticle"]

g = {
    "PMID": [str(get(r,"MedlineCitation.PMID")) for r in articles],
    "BACKGROUND": [get_segment(r, "BACKGROUND") for r in articles],
    "OBJECTIVE": [get_segment(r, "OBJECTIVE") for r in articles],
    "METHODS": [get_segment(r, "METHODS") for r in articles],
    "RESULTS": [get_segment(r, "RESULTS") for r in articles],
    "CONCLUSIONS": [get_segment(r, "CONCLUSIONS") for r in articles]
}


df2 = pd.DataFrame(g)
df2["PMID"] = df2["PMID"].astype(int)
dfx = pd.merge(df, df2, on="PMID")
dfx.to_csv("enriched_pm_10k.csv")


#dfx.head(), len(df), len(df2), df[:6768]["PMID"][0], df2[:6768]["PMID"][0]

(       PMID                                              Title  \
 0  28946522  The effect of manual therapy and exercise in p...   
 1  28325149  Low Dose Naltrexone in the Treatment of Fibrom...   
 2  29514165  High-Dose Erythropoietin for Asphyxia and Ence...   
 3  29424396  [Quality of sleep and academic performance in ...   
 4  29754671  Rationale and design for the Vascular Outcomes...   
 
                                              Authors  \
 0                 Ulger O, Demirel A, Oz M, Tamer S.   
 1  Metyas S, Chen CL, Yeter K, Solyman J, Arkfeld...   
 2  Juul SE, Comstock BA, Heagerty PJ, Mayock DE, ...   
 3  Bugueño M, Curihual C, Olivares P, Wallace J, ...   
 4  Capell WH, Bonaca MP, Nehler MR, Chen E, Kitte...   
 
                                             Citation First Author  \
 0  J Back Musculoskelet Rehabil. 2017 Nov 6;30(6)...      Ulger O   
 1  Curr Rheumatol Rev. 2018;14(2):177-180. doi: 1...     Metyas S   
 2  Neonatology. 2018;113(4):331-338. doi:

In [63]:
#export
def get_first(p):
    return get(split_into_sentences(p), 0)
#get(split_into_sentences(t), 0) for t in dfx["METHODS"][:10] if split_into_sentences(t))

In [88]:
#reformating data
dfy = pd.DataFrame([], columns=["PMID", "Title", "source", "DOI", "text"])
tags = ["OBJECTIVE", "METHODS", "RESULTS", "CONCLUSIONS"]
for tag in tags:
    df = dfx[dfx[tag] != "None"]
    df["paragraph"] = df[tag]
    df["text"] = dfx[tag].map(get_first)
    df["source"] = "pm_" + tag
    df = df[dfy.columns]
    dfy = dfy.append(df)

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [89]:
dfy = dfy.dropna(subset=["text"])
dfy["labels"] = "0"

In [91]:
dfy.to_csv("downloads/struct_abstracts_negatives_18k.csv")

### Fetching dataset from Google Drive


In [60]:
path = 'https://drive.google.com/uc?export=download&id='
df1 = pd.read_csv(path + "1t2m6IxieiE0hZ8TcLJe9foc7upngrA_0")
df1["source"] = " limitation|problem ... with ... l=be that"

In [61]:
df2 = pd.read_csv(path + "1t-2rU83KdppSrwDnqdQUi6Pvtw13D0bQ")
df2["source"] = "a ... drawback of"
df3 = pd.read_csv(path + "1iGdfLJ4MkTO8oOomFJ84JCc7nf2zYBul")
df3["source"] = "a {negative_adjectives} ...  {problem_nouns}"

In [62]:
len(df1), len(df2), len(df3)

(947, 1455, 18573)

In [63]:
dff = df1.append(df2).append(df3)
dff.head()

,sentence_id,title,article_link,sentence_text,year,mesh_list,journal_title,volume,issue,abstract,source,paragraph_text
0,1263595,"Shoe-Integrated, Force Sensor Design for Conti...",https://pubmed.ncbi.nlm.nih.gov/32545528/,One problem with such methods is that they tap...,2020,"['Biomechanical Phenomena', 'Body Weight', 'Eq...","Sensors (Basel, Switzerland)",20,12,Traditional pedobarography methods use direct ...,limitation|problem ... with ... l=be that,NaN
1,1821146,The Problem with Gout Is That It's Still Such ...,https://pubmed.ncbi.nlm.nih.gov/27481987/,The Problem with Gout Is That It 's Still Such...,2016,"['Allopurinol', 'therapeutic use', 'Arthritis,...",The Journal of rheumatology,43,8,NaN,limitation|problem ... with ... l=be that,NaN
2,1840641,Generation of noble-gas binding sites for crys...,https://pubmed.ncbi.nlm.nih.gov/11752783/,One problem with this approach is that not all...,2002,"['Bacteriophage T4', 'enzymology', 'Binding Si...","Acta crystallographica. Section D, Biological ...",58,Pt 1,"In recent years, the use of noble-gas complexe...",limitation|problem ... with ... l=be that,NaN
3,2187391,"Segmentation of Crohn, Lymphangiectasia, Xanth...",https://pubmed.ncbi.nlm.nih.gov/22874169/,"However , a tough problem associated with this...",2012,"['Algorithms', 'Artificial Intelligence', 'Cap...",Studies in health technology and informatics,180,NaN,Wireless capsule endoscopy (WCE) is a great br...,limitation|problem ... with ... l=be that,NaN
4,3202138,Vection in virtual reality modulates vestibula...,https://pubmed.ncbi.nlm.nih.gov/31233640/,While significant technological advancements a...,2019,"['Female', 'Humans', 'Male', 'Muscle, Skeletal...",REFERENCES,50,10,The popularity of virtual reality (VR) has inc...,limitation|problem ... with ... l=be that,NaN


In [64]:
dff["PMID"] = dff["article_link"].map(lambda t: t.split("/")[-2])

In [65]:
len(dff.drop_duplicates(subset=["PMID"])), len(dff)

(20822, 20975)

In [66]:
dff = dff.drop_duplicates(subset=["PMID"])

In [67]:
dff = dff.rename(columns={'sentence_text': "text", "title": "Title"}, errors="raise")

In [68]:
dff

,sentence_id,Title,article_link,text,year,mesh_list,journal_title,volume,issue,abstract,source,paragraph_text,PMID
0,1263595,"Shoe-Integrated, Force Sensor Design for Conti...",https://pubmed.ncbi.nlm.nih.gov/32545528/,One problem with such methods is that they tap...,2020,"['Biomechanical Phenomena', 'Body Weight', 'Eq...","Sensors (Basel, Switzerland)",20,12,Traditional pedobarography methods use direct ...,limitation|problem ... with ... l=be that,NaN,32545528
1,1821146,The Problem with Gout Is That It's Still Such ...,https://pubmed.ncbi.nlm.nih.gov/27481987/,The Problem with Gout Is That It 's Still Such...,2016,"['Allopurinol', 'therapeutic use', 'Arthritis,...",The Journal of rheumatology,43,8,NaN,limitation|problem ... with ... l=be that,NaN,27481987
2,1840641,Generation of noble-gas binding sites for crys...,https://pubmed.ncbi.nlm.nih.gov/11752783/,One problem with this approach is that not all...,2002,"['Bacteriophage T4', 'enzymology', 'Binding Si...","Acta crystallographica. Section D, Biological ...",58,Pt 1,"In recent years, the use of noble-gas complexe...",limitation|problem ... with ... l=be that,NaN,11752783
3,2187391,"Segmentation of Crohn, Lymphangiectasia, Xanth...",https://pubmed.ncbi.nlm.nih.gov/22874169/,"However , a tough problem associated with this...",2012,"['Algorithms', 'Artificial Intelligence', 'Cap...",Studies in health technology and informatics,180,NaN,Wireless capsule endoscopy (WCE) is a great br...,limitation|problem ... with ... l=be that,NaN,22874169
4,3202138,Vection in virtual reality modulates vestibula...,https://pubmed.ncbi.nlm.nih.gov/31233640/,While significant technological advancements a...,2019,"['Female', 'Humans', 'Male', 'Muscle, Skeletal...",REFERENCES,50,10,The popularity of virtual reality (VR) has inc...,limitation|problem ... with ... l=be that,NaN,31233640
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18568,362702792,Falls and injuries in frail and vigorous commu...,https://pubmed.ncbi.nlm.nih.gov/1987256/,These findings suggest that fall-related injur...,1991,"['Accidental Falls', 'prevention & control', '...",Journal of the American Geriatrics Society,39,1,Identification of different types of falls and...,a {negative_adjectives} ... {problem_nouns},NaN,1987256
18569,362714838,Intermittent catheterization: sterile or clean?,https://pubmed.ncbi.nlm.nih.gov/1989043/,Bacteriuria -- asymptomatic and symptomatic --...,1991,"['Bacteriuria', 'etiology', 'nursing', 'preven...",Rehabilitation nursing : the official journal ...,16,1,Bacteriuria--asymptomatic and symptomatic--alw...,a {negative_adjectives} ... {problem_nouns},NaN,1989043
18570,362715256,Variceal rebleeding after portosystemic shunti...,https://pubmed.ncbi.nlm.nih.gov/1989102/,Strategies and solutions to a vexing problem .,1991,"['Combined Modality Therapy', 'Esophageal and ...",The Surgical clinics of North America,71,1,The purpose of this review was to discuss an a...,a {negative_adjectives} ... {problem_nouns},NaN,1989102
18571,362724620,Neuroprotective Effect of Natural Products on ...,https://pubmed.ncbi.nlm.nih.gov/26645998/,Peripheral nerve injury ( PNI ) is a serious p...,2016,"['Animals', 'Biological Products', 'therapeuti...",Neurochemical research,41,4,Peripheral nerve injury (PNI) is a serious pub...,a {negative_adjectives} ... {problem_nouns},NaN,26645998


In [69]:
dff = dff[['Title',"text", 'mesh_list', 'journal_title', 'source', 'PMID']]

In [70]:
dff["labels"] = "1"

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
